Let's fetch data from the internet to csv files

In [16]:
import requests

def fetch_air_range(station_id, date_from, date_until):
    url = 'http://airviro.klab.ee/station/csv'
    data = {
        'filter[type]': 'INDICATOR',
        'filter[cancelSearch]': '',
        'filter[stationId]': station_id,
        'filter[dateFrom]': date_from,
        'filter[dateUntil]': date_until,
        'filter[submitHit]': '1',
        'filter[indicatorIds]': ''
    }
    response = requests.post(url, data)
    return response.text

import datetime

def get_first_and_last_day_of_month(year, month):
    # Get the first day of the month
    first_day = datetime.date(year, month, 1)

    # Get the number of days in the month
    if month == 12:
        num_days = 31
    else:
        num_days = (datetime.date(year, month+1, 1) - datetime.timedelta(days=1)).day

    # Get the last day of the month
    last_day = datetime.date(year, month, num_days)

    return first_day, last_day

for month in range(1, 13):
    first_day, last_day = get_first_and_last_day_of_month(2022, month)
    print(first_day, last_day)

    data = fetch_air_range(8, first_day, last_day)
    with open(f'air_data/air_{month}.csv', 'w', encoding='utf-8-sig') as f:
        f.write(data)

#response = requests.post(url, data)
#print(response.text)


#print(response.status_code)

2022-01-01 2022-01-31
2022-02-01 2022-02-28
2022-03-01 2022-03-31
2022-04-01 2022-04-30
2022-05-01 2022-05-31
2022-06-01 2022-06-30
2022-07-01 2022-07-31
2022-08-01 2022-08-31
2022-09-01 2022-09-30
2022-10-01 2022-10-31
2022-11-01 2022-11-30
2022-12-01 2022-12-31


Let's join the csv files into one csv file

In [18]:
data = []
for month in range (1,13):
    with open(f'air_data/air_{month}.csv', 'r') as f:
        if month == 1:
            header = f.readline()
            data.append(header)
        else:
            f.readline()
        data.extend(f.readlines())

with open('air_data/air_2022.csv', 'w') as f:
    f.writelines(data)

After using OpenRefine, lets create sqlite database with one main table and create three new tables there

In [1]:
import sqlite3

conn = sqlite3.connect('air-2022.db')

cur = conn.cursor()
with open('air-2022-csv.sql', 'r') as file:
    script = file.read()
cur.executescript(script)

cur2 = conn.cursor()
with open('air-2022-tables.sql', 'r') as file:
    script2 = file.read()
cur2.executescript(script2)

conn.commit()
conn.close()

In [5]:

import pandas as pd
conn = sqlite3.connect('air-2022.db')
df = pd.read_sql_query('SELECT * FROM air_2022_hour_avg', conn)
conn.close()

df.head()



,hour,AVG(SO2),AVG(NO2),AVG(CO),AVG(O3),AVG(PM10),AVG(PM2_5),AVG(TEMP),AVG(WD10),AVG(WS10)
0,00,0.343526,9.193388,0.283534,45.368956,17.073352,7.806044,6.486849,199.296703,1.048077
1,01,0.326171,7.773901,0.263973,44.657143,15.951233,7.135890,6.076712,202.243836,1.049041
2,02,0.302479,6.488430,0.248055,43.889286,14.187945,6.292877,5.718356,197.643836,1.072603
3,03,0.277901,5.573130,0.235247,43.121212,13.444505,6.027273,5.432967,194.395604,1.048077
4,04,0.267403,4.706111,0.223407,42.524518,12.227198,5.636364,5.180220,196.912088,1.033242
